# House Prices: Advanced Regression Techniques

## Missing values

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

In [2]:
house_prices = pd.read_csv('../train.csv')

In [3]:
house_prices.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [4]:
missing = house_prices.isna().sum()
missing[missing > 0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [5]:
house_prices.loc[house_prices['BsmtFinSF1'] == 0 ,'BsmtFinType1'] = 'Unf'
house_prices.loc[house_prices['BsmtFinSF2'] == 0 ,'BsmtFinType2'] = 'Unf'

In [6]:
house_prices.loc[house_prices['GarageYrBlt'].isna(), 'GarageYrBlt'] = house_prices.loc[house_prices['GarageYrBlt'].isna(), 'YearBuilt']

In [7]:
# 1 missing value
house_prices.loc[house_prices['Electrical'].isna(), 'Electrical'] = 'SBrkr'

In [8]:
house_prices.loc[house_prices['MasVnrType'].isna(), 'MasVnrType'] = 'None'
house_prices.loc[house_prices['MasVnrArea'].isna(), 'MasVnrArea'] = 0

In [9]:
possible_lotFrontage_values = house_prices.groupby(['LotConfig', 'LotShape']).agg({'LotFrontage':'median'})
possible_lotFrontage_values = possible_lotFrontage_values.reset_index()
possible_lotFrontage_values

,LotConfig,LotShape,LotFrontage
0,Corner,IR1,96.0
1,Corner,IR2,96.0
2,Corner,IR3,247.5
3,Corner,Reg,75.0
4,CulDSac,IR1,50.0
5,CulDSac,IR2,49.0
6,CulDSac,IR3,168.0
7,CulDSac,Reg,NaN
8,FR2,IR1,77.0
9,FR2,IR2,48.0


In [10]:
missing_values = house_prices.loc[house_prices['LotFrontage'].isna(), ['LotConfig', 'LotShape']]
missing_values.shape

(259, 2)

In [11]:
merge_missing_with_domain = pd.merge(missing_values, possible_lotFrontage_values, how='left', left_on=['LotConfig', 'LotShape'], right_on=['LotConfig', 'LotShape'])
merge_missing_with_domain.shape

(259, 3)

In [12]:
house_prices.loc[house_prices['LotFrontage'].isna(), 'LotFrontage'] = merge_missing_with_domain['LotFrontage'].values

In [13]:
house_prices.loc[house_prices['LotFrontage'].isna(), 'LotFrontage']

660   NaN
Name: LotFrontage, dtype: float64

In [14]:
possible_lotFrontage_values = house_prices.groupby(['LotConfig']).agg({'LotFrontage':'median'})
possible_lotFrontage_values = possible_lotFrontage_values.reset_index()
possible_lotFrontage_values

,LotConfig,LotFrontage
0,Corner,89.0
1,CulDSac,50.0
2,FR2,65.0
3,FR3,59.5
4,Inside,68.0


In [15]:
missing_values = house_prices.loc[house_prices['LotFrontage'].isna(), ['LotConfig', 'LotShape']].copy()
missing_values.shape

(1, 2)

In [16]:
merge_missing_with_domain = pd.merge(missing_values, possible_lotFrontage_values, how='left', left_on=['LotConfig'], right_on=['LotConfig'])
merge_missing_with_domain

,LotConfig,LotShape,LotFrontage
0,CulDSac,Reg,50.0


In [17]:
house_prices.loc[house_prices['LotFrontage'].isna(), 'LotFrontage'] = merge_missing_with_domain['LotFrontage'].values

In [18]:
house_prices.loc[house_prices['LotFrontage'].isna(), 'LotFrontage']

Series([], Name: LotFrontage, dtype: float64)

In [19]:
missing = house_prices.isna().sum()
missing[missing > 0].index

Index(['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2',
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PoolQC', 'Fence', 'MiscFeature'],
      dtype='object')

In [20]:
house_prices.drop(columns=['Alley', 'BsmtCond', 'BsmtFinType2', 'GarageQual', 'GarageCond',
       'PoolQC', 'Fence', 'MiscFeature', 'Id'], inplace=True)

In [21]:
house_prices.fillna('No', inplace=True)

In [22]:
house_prices.to_csv('../train_wo_missing_values.csv', index=False)